# Data dictionary

- V6 is heading in degrees
- V2 seems to change for each aircraft during exit-entry
- V14 is 'from' airport
- V7 is altitude in ft
- V8 is ground speed in kts
- V15 is 'to' airport
- V3 looks like a proxy for aircraft
- V13 is timestamp
- V18 is instantaneous climb rate (jittery)
- V16_V19 looks like a good proxy for aircraft id: more relevant than V3 (aircraft)
- V16_V19 can refine V3
- V17 looks like 'aircraft at airport' flag

# Note

Plotting fails because rnaturalearth could not be installed

# Initialization

In [1]:
lower_lat <- 35
upper_lat <- 45
lower_lon <- -78
upper_lon <- -68
lat_range <- 0.4
lon_range <- 0.5
ts_diff_ground <- 1080
ts_diff_exit_enter <- 1080
ts_diff_expired <- 7200
airport_lat_half_range <- airport_lon_half_range <- 0.1

In [2]:
suppressMessages({
    library(plyr)
    library(dplyr)
    library(data.table)
    library(bit64)
})

In [3]:
########################################################################
############## Reading ADS-B data and preprocessing ####################
########################################################################

landed_altitude <- 1000 # may not need this because of V17
v1_ground <- 50
surely_grounded_speed <- 10
ac_data <- fread("../../adsb_fetch/ads-b_with_id.csv")
ac_data <- rbind(ac_data, fread("../../adsb_fetch/ads-b_exta_with_id_no_est.csv"))
ac_data <- ac_data %>% rename(azimuth = V6, source = V14, destination = V15, altitude = V7,
                              ground_speed = V8, aircraft = V3, ts = V13, climb_rate = V18,
                              ground_flag = V17, lat = V4, lon = V5, aircraft_type = V11)

# New: Ground flag and altitude update to avoid discrepancies
ac_data$ground_flag[ac_data$altitude == 0] <- 1
ac_data$altitude[ac_data$ground_flag == 1] <- 0

ac_data$altitude <- 100 * ceiling(ac_data$altitude/100)
ac_data <- ac_data[ac_data$V10 != "F-EST"] # Removing estimated positions
setorder(ac_data, aircraft, ts)
ac_data <- ac_data[, .SD[1], by = list(aircraft, ts)]

ac_data$ts_readable <- as.POSIXct.numeric(as.numeric(ac_data$ts), origin="1970-01-01")

In [4]:
# Removing anomalous aircraft
ac_data$quadrant <- ceiling(ac_data$azimuth/90)
ac_data <- ac_data[!grepl(ac_data$aircraft, pattern = "^FAA"), ]
# ac_to_remove <- data.frame(aircraft = unique(ac_data$aircraft[ac_data$altitude >= 30000 & ac_data$ground_speed < 100]))

In [5]:
ac_data$V16_V19 <- paste0(ac_data$V16, "_", ac_data$V19)
ac_data[, c("last_lat", "last_lon", "last_azimuth", "last_altitude",
            "last_ts", "last_source", "last_destination", "last_V9",
            "last_ground_flag", "last_V16_V19", "last_quadrant",
            "last_ground_speed", "last_V2",
            "next_lat", "next_lon", "next_azimuth", "next_altitude",
            "next_ts", "next_source", "next_destination", "next_V9",
            "next_ground_flag", "next_V16_V19", "next_quadrant",
            "next_ground_speed", "next_V2",
            "uniq_source",
            "uniq_destination",
            "max_ac_alt") :=
                list(c(lat[1], lat[-.N]), c(lon[1], lon[-.N]), c(azimuth[1], azimuth[-.N]), c(altitude[1], altitude[-.N]),
                     c(ts[1], ts[-.N]), c(NA, source[-.N]), c(NA, destination[-.N]), c(NA, V9[-.N]),
                     c(ground_flag[1], ground_flag[-.N]), c(V16_V19[1], V16_V19[-.N]), c(quadrant[1], quadrant[-.N]),
                     c(ground_speed[1], ground_speed[-.N]), c(V2[1], V2[-.N]),
                     c(lat[-.1], NA), c(lon[-1], NA), c(azimuth[-.1], NA), c(altitude[-1], NA),
                     c(ts[-1], NA), c(source[-1], NA), c(destination[-1], NA), c(V9[-1], NA),
                     c(ground_flag[-1], NA), c(V16_V19[-1], NA), c(quadrant[-1], NA),
                     c(ground_speed[-1], NA), c(V2[-1], NA),
                     paste0(sort(unique(source)), collapse = " "),
                     paste0(sort(unique(destination)), collapse = " "),
                     max(altitude)), by = "aircraft"]
ac_data$last_ts_diff <- ac_data$ts - ac_data$last_ts
ac_data$next_ts_diff <- ac_data$next_ts - ac_data$ts
ac_data$destination <- toupper(ac_data$destination)
ac_data$est_climb_rate <- 60 * (as.numeric(ac_data$altitude) - as.numeric(ac_data$last_altitude))/as.numeric(ac_data$last_ts_diff) # ft/min
ac_data$climb_rate_ratio <- 1
condition <- ac_data$est_climb_rate != 0 & !is.na(ac_data$est_climb_rate) & ac_data$last_ts_diff < 1200
ac_data$climb_rate_ratio[condition] <- ac_data$climb_rate[condition]/ac_data$est_climb_rate[condition]
ac_data$quadrant_change <- ac_data$quadrant != ac_data$last_quadrant

In [6]:
########################################################################
################# Reading airports data and checking ###################
########################################################################

# This file seems to be more reliable
airports <- read.csv("../../../GlobalAirportDatabase.txt", header = F, stringsAsFactors = F)
airports <- airports %>% dplyr::rename(airport = V5, lat = V7, lon = V8)
airports$airport <- toupper(airports$airport)
airports <- airports[airports$airport != "", ]

# 511 of the 563 destinations can be matched

airports <- airports[data.table::between(airports$lat, lower = lower_lat, upper = upper_lat) &
                       data.table::between(airports$lon, lower = lower_lon, upper = upper_lon), ]
rownames(airports) <- airports$V1 <- NULL

In [7]:
table(ac_data$altitude[ac_data$ground_flag == 1])


      0 
8699091 

In [8]:
########################################################################
############### Defining landing and entry into sector #################
########################################################################

ac_data$touchdown <- (ac_data$last_ground_flag == 0) &
  (ac_data$ground_flag == 1)
ac_data[, "last_touchdown" := c(F, touchdown[-.N]), by = aircraft]
ac_data$touchdown <- ac_data$last_touchdown
ac_data$last_touchdown <- NULL
# ac_data[, "touchdown_id" := cumsum(last_touchdown), by = aircraft]
# ac_data[, "max_touchdown" := max(touchdown_id), by = aircraft]
# ac_data[, max_altitude_touchdown_id := max(altitude), by = list(aircraft, touchdown_id)]
# print(sum(ac_data$last_touchdown & (ac_data$max_altitude_touchdown_id < 0.1 * ac_data$max_ac_alt)))
# ac_data$last_touchdown[ac_data$last_touchdown & (ac_data$max_altitude_touchdown_id < 0.1 * ac_data$max_ac_alt)] <- F
# ac_data[, "touchdown_id" := cumsum(touchdown), by = aircraft]
# ac_data[, "max_touchdown" := max(touchdown_id), by = aircraft]
# ac_data[, max_altitude_touchdown_id := max(altitude), by = list(aircraft, touchdown_id)]

# Consecutive landings within 10 mins is not possible
# These can be considered as missed approaches, therefore only the final landing is considered
# false_idx <- c()
# for(ac in unique(ac_data$aircraft[ac_data$touchdown & ac_data$max_touchdown > 1])) {
#   ac_df <- ac_data[ac_data$aircraft == ac & ac_data$touchdown, ]
#   time_between_touchdowns <- as.numeric(difftime(ac_df$ts_readable[-1], ac_df$ts_readable[-nrow(ac_df)], units = "s"))
#   if(any(time_between_touchdowns < 600)) {
#     idx1 <- which(time_between_touchdowns < 600)
#     ac_data_idx <- which(ac_data$aircraft == ac & ac_data$touchdown)
#     false_idx <- c(false_idx, ac_data_idx[idx1])
#   }
# }
# if(length(false_idx) > 0) {
#   ac_data$touchdown[false_idx] <- F
# }

In [9]:
# touchdown_df <- ac_data[ac_data$touchdown, c("aircraft", "ts")]
# rows = nrow(touchdown_df) + 1

# while(rows != nrow(touchdown_df)) {
#     rows <- nrow(touchdown_df)
#     print(rows)
#     touchdown_df[, "next_touchdown_ts" := c(ts[-1], NA), by = "aircraft"]
#     touchdown_df$touchdown_time_diff <- touchdown_df$next_touchdown_ts - touchdown_df$ts
#     touchdown_df$touchdown_time_diff[is.na(touchdown_df$touchdown_time_diff)] <- 600
#     touchdown_df <- touchdown_df[touchdown_df$touchdown_time_diff >= 600, ]
# }
# touchdown_df$touchdown <- T
# ac_data$touchdown <- NULL
# ac_data <- merge(ac_data, touchdown_df, all.x = T, all.y = T)
# ac_data$touchdown[is.na(ac_data$touchdown)] <- F
# sum(ac_data$touchdown)

In [10]:
########################################################################
################## Testing definition of touchdown #####################
########################################################################

identify_airport <- function(lat, lon, lat_half_range = airport_lat_half_range,
                             lon_half_range = airport_lon_half_range) {
  df <- airports[data.table::between(airports$lat, lower = lat - lat_half_range,
                                     upper = lat + lat_half_range) &
                   data.table::between(airports$lon, lower = lon - lon_half_range,
                                       upper = lon + lon_half_range), ]
  if(nrow(df) > 1) {
    dists <- apply(df, 1, function(row) {
      row["lat"] <- as.numeric(row["lat"])
      row["lon"] <- as.numeric(row["lon"])
      return(distm(x = c(row["lon"], row["lat"]), y = c(lon, lat)))
    })
    df <- df[which.min(dists), ]
    df$dist <- min(dists)
  } else if(nrow(df) == 1) {
    df$dist <- 0
  } else {
    dists <- apply(airports, 1, function(row) {
      row["lat"] <- as.numeric(row["lat"])
      row["lon"] <- as.numeric(row["lon"])
      return(distm(x = c(lon, lat), y = c(row["lon"], row["lat"])))
    })
    df <- airports[which.min(dists), ]
    df$dist <- min(dists)
  }
  return(df[, c("airport", "dist")])
}


touchdown <- ac_data[ac_data$touchdown, ]
landed_airport <- list()
library(geosphere)
for(i in 1:nrow(touchdown)) {
  landed_airport[[i]] <- identify_airport(touchdown$lat[i], touchdown$lon[i])
}
all(sapply(landed_airport, nrow) == 1)
condition <- sapply(landed_airport, function(row) row$dist) <= 10000
print(mean(condition)) # landed location is within 10 km of airport
touchdown_idx <- which(ac_data$touchdown)
non_touchdown <- touchdown_idx[!condition]
ac_data$touchdown[non_touchdown] <- F
ac_data$airport <- ""
ac_data$airport[ac_data$touchdown] <- sapply(landed_airport, function(row) row$airport)[condition]

[1] TRUE

[1] 0.963998


In [11]:
precision_recall <- F
if(precision_recall) {
  dests <- ac_data$last_destination[ac_data$touchdown & ac_data$last_destination != ""]
  print(mean(dests == ac_data$airport[ac_data$touchdown & ac_data$last_destination != ""]))
  # Accuracy of 'given' destination: 0.9568
  # Actually the precision is 0.9635 (from azimuth analysis)
  
  # JFK recall (assuming 'airport' is true value)
  tbl <- table(ac_data$destination[ac_data$touchdown & ac_data$airport == "JFK"])
  print(sort(tbl, decreasing = T)/sum(tbl))
  # JFK recall is 0.9177, "" is the main issue
}

In [12]:
lat_lon_boundary_flag <- function(lt, ln, last_lt, last_ln,
                                  lower_lt = lower_lat, lower_ln = lower_lon, upper_lt = upper_lat,
                                  upper_ln = upper_lon, lt_range = lat_range, ln_range = lon_range) {
  ans <- (ln < ((lower_ln + ln_range) | last_ln < (lower_ln + ln_range))) |
    ((ln > (upper_ln - ln_range) | last_ln > (upper_ln - ln_range))) |
    ((lt < (lower_lt + lt_range) | last_ln < (lower_lt + lt_range))) |
    ((lt > (upper_lt - lt_range) | last_ln > (upper_lt - lt_range)))
  return(ans)
}


ac_data$landed <- ac_data$touchdown
ac_data$entered <- (ac_data$V2 != ac_data$last_V2) |
  (lat_lon_boundary_flag(ac_data$lat, ac_data$lon, ac_data$last_lat, ac_data$last_lon) &
          ac_data$last_ts_diff > 1080)

ac_data$entered[is.na(ac_data$entered)] <- F
ac_data$expired <- (ac_data$last_ts_diff > ts_diff_expired)
ac_data$landed_or_entered_or_expired <- ac_data$landed | ac_data$entered | ac_data$expired
ac_data[, "num_id" := cumsum(landed_or_entered_or_expired), by = "aircraft"]
table(ac_data$num_id)
ac_data$id <- paste0(ac_data$aircraft, "_", ac_data$num_id)


      0       1       2       3       4       5       6       7       8       9 
3758765 2114839 2702021 2194311 1623589 1795420 1570134 1365213 1396973 1240793 
     10      11      12      13      14      15      16      17      18      19 
1110341 1092417 1030091  932642  896434  870288  781884  750496  739812  688699 
     20      21      22      23      24      25      26      27      28      29 
 654600  643991  595907  586151  545781  498007  476803  448553  433851  410152 
     30      31      32      33      34      35      36      37      38      39 
 397044  384054  355629  343815  331523  303632  293041  293737  269333  265837 
     40      41      42      43      44      45      46      47      48      49 
 256011  245934  241916  228248  221145  201703  208786  193281  195236  193551 
     50      51      52      53      54      55      56      57      58      59 
 177487  180472  173159  163675  159082  153741  155119  140112  137516  134366 
     60      61      62    

In [13]:
########################################################################
############### Plotting trajectory of resulting IDs ###################
########################################################################

plotting <- F

if(plotting) {
  dir.create("jfk_landing", showWarnings = F)
  setwd("jfk_landing")
  visualize_map <- function(ac_df, color_var = "id", sampl = T,
                            min_lon = NULL, max_lon = NULL, min_lat = NULL, max_lat = NULL) {
    ac_df$num_id <- as.integer(as.factor(ac_df$id))
    ac_df <- split(ac_df, ac_df$num_id)
    if(sampl)
      ac_df <- ac_df[sample(x = 1:length(ac_df), size = n_aircraft, replace = F)]
    ac_df <- do.call(rbind, ac_df)
    ac_df <- ac_df[order(ac_df$ts), ]
    p <- ggplot() +
      geom_sf(data = world) +
      coord_sf(xlim = c(min_lon, max_lon), ylim = c(min_lat, max_lat), expand = F) +
      geom_path(data = ac_df, arrow = arrow(type = "closed", angle = 18,
                                            length = unit(0.1, "inches")),
                aes(x = lon, y = lat, frame = ts, color = id)) +
      xlim(c(min_lon, max_lon)) +
      ylim(c(min_lat, max_lat))
    return(p)
  }
  
  ac_data[, c("last_entered", "next_entered",
              "last_airport", "next_airport",
              "last_touchdown", "next_touchdown") :=
            list(c(F, entered[-.N]), c(entered[-1], F),
                 c(airport[1], airport[-.N]), c(airport[-1], NA),
                 c(F, touchdown[-.N]), c(touchdown[-1], F)), by = "aircraft"]
  jfk_data <- data.frame(id = ac_data$id[ac_data$next_airport == "JFK" & ac_data$next_touchdown
                                         & ac_data$source != "JFK"], jfk_landing_flag = T)
  jfk_data <- merge(jfk_data, ac_data, all.x = T, all.y = T)
  jfk_data$jfk_landing_flag[is.na(jfk_data$jfk_landing_flag)] <- F
  
  min_lon <- min(jfk_data$lon)
  max_lon <- max(jfk_data$lon)
  min_lat <- min(jfk_data$lat)
  max_lat <- max(jfk_data$lat)
  uniq_id <- unique(jfk_data$aircraft)
  
  
  library(parallel)
  library(ggplot2)
  library(rnaturalearth)
  cores <- detectCores() - 1
  cl <- parallel::makeCluster(cores)
  split_num <- ceiling(length(uniq_id)/cores)
  num_id <- ceiling((1:length(uniq_id))/split_num)
  id_splits <- split(uniq_id, num_id)
  jfk_data1 <- jfk_data[, c("id", "ts", "lon", "lat", "aircraft")]
  world <- ne_countries(scale = "medium", returnclass = "sf")
  clusterExport(cl, list("jfk_data1", "coord_sf", "geom_sf", "world", "visualize_map",
                         "min_lon", "max_lon", "min_lat", "max_lat", "png", "paste0",
                         "print", "dev.off", "ggplot", "geom_path", "arrow", "aes",
                         "unit", "xlim", "ylim"))
  
  plot_ids <- function(uniq_id) {
#     setwd("~/Desktop/Coding/Old/ATC_Analysis_2019_04_01/Refedined_Landing/jfk_landing/")
    for(id1 in uniq_id) {
      ac_df <- jfk_data1[jfk_data1$aircraft == id1, ]
      p <- visualize_map(ac_df, sampl = F, min_lon = min_lon,
                         max_lon = max_lon, min_lat = min_lat,
                         max_lat = max_lat)
      png(paste0(id1, ".png"), width = 1366, height = 768)
      print(p)
      dev.off()
    }
    setwd("../")
  }
  
  parLapply(cl, id_splits, plot_ids)
  stopCluster(cl)
  
  setwd("../")
}

In [14]:
########################################################################
################## Saving final prepropcessed data #####################
########################################################################

saveRDS(ac_data[, c("aircraft", "id", "ts", "ts_readable", "lon", "lat", "altitude", "ground_speed", "azimuth", "source", "destination")], "ac_data_processed.Rds")
setorder(ac_data, aircraft, ts)
ac_data[, c("last_touchdown", "next_touchdown", "last_entered", "next_entered", "last_airport", "next_airport") :=
          list(c(F, touchdown[-.N]), c(touchdown[-1], F), c(F, entered[-.N]), c(entered[-1], F), c("", airport[-.N]), c(airport[-1], "")), by = "aircraft"]
ac_data$next_touchdown[is.na(ac_data$next_touchdown)] <- F
jfk_data <- data.frame(id = unique(ac_data$id[ac_data$next_airport == "JFK" & ac_data$next_touchdown &
                                              ac_data$source != "JFK"]), jfk_landing_flag = T)
jfk_data <- merge(jfk_data, ac_data, all.x = T, all.y = T)
jfk_data$jfk_landing_flag[is.na(jfk_data$jfk_landing_flag)] <- F
setDT(jfk_data)
jfk_data[, "max_id_alt" := max(altitude), by = "id"]
write.csv(jfk_data[, c("id", "aircraft", "lat", "lon", "ts", "altitude", "climb_rate",
                       "azimuth", "ground_speed", "jfk_landing_flag", "source",
                       "destination", "ground_flag")],
          "../../../samples_with_jfk_landing_flag.csv", row.names = F)
saveRDS(ac_data, "full_ac_data.Rds")
# saveRDS(jfk_data[jfk_data$max_id_alt >= 20000, ], "high_ac_data.Rds")
saveRDS(jfk_data, "full_jfk_data.Rds")

In [20]:
length(unique(jfk_data$id))

[1] 238829